Importing the required Modules:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder

Reading in the dataset as a Pandas Dataframe:


In [ ]:
df=pd.read_csv("superstore.csv")
tempdf=df.copy()
coor= df.copy()

In [ ]:
# Separate numerical and categorical columns
numerical_columns = coor.select_dtypes(include=['float64','int64','int32']).columns
categorical_columns = coor.select_dtypes(include=['object', 'category']).columns

# Calculate correlation matrix for numerical columns
numerical_corr = coor[numerical_columns].corr()

# Calculate correlation matrix for categorical columns
#categorical_corr = tempdf[categorical_columns].corr()

# Print correlation matrices
print("Correlation Matrix for Numerical Columns:")
numerical_corr


In [ ]:
# Separate numerical and categorical columns
numerical_columns = coor.select_dtypes(include=['float64','int64','int32']).columns
categorical_columns = coor.select_dtypes(include=['object', 'category']).columns

# Calculate correlation matrix for numerical columns
numerical_corr = coor[numerical_columns].corr()

# Calculate correlation matrix for categorical columns
#categorical_corr = tempdf[categorical_columns].corr()

# Print correlation matrices
print("Correlation Matrix for Numerical Columns:")
numerical_corr


Removing Miscellaneous(Unwanted) columns:

In [4]:
tempdf.pop("Unnamed: 0") # Dosen't provide any useful information
tempdf.pop("记录数") # Dosen't provide any useful information
tempdf.pop("Row.ID") # Reduntant column that provides the row number
tempdf.pop('Year')
tempdf.pop('Market2')
tempdf.pop('Sub.Category')
print(tempdf.columns)

Index(['Unnamed: 0', 'Category', 'City', 'Country', 'Customer.ID',
       'Customer.Name', 'Discount', 'Market', '记录数', 'Order.Date', 'Order.ID',
       'Order.Priority', 'Product.ID', 'Product.Name', 'Profit', 'Quantity',
       'Region', 'Row.ID', 'Sales', 'Segment', 'Ship.Date', 'Ship.Mode',
       'Shipping.Cost', 'State', 'Sub.Category', 'Year', 'Market2', 'weeknum'],
      dtype='object')


0        36624.0
1        37033.0
2        31468.0
3        31469.0
4        32440.0
          ...   
51285    33646.0
51286    33645.0
51287    32321.0
51288    35917.0
51289    37371.0
Name: Row.ID, Length: 51290, dtype: float64

Rearraging the columns for convenience

In [5]:
columns = [
    # Order Details
    "Order.ID", "Order.Date", "weeknum", "Ship.Date", "Order.Priority", "Ship.Mode", "Shipping.Cost",
    # Customer Details
    "Customer.ID", "Customer.Name", "Segment",    
    # Location Details
    "City", "State", "Country","Region",   
    # Product Details
    "Product.ID", "Product.Name", "Category",    
    # Sales and Financials
    "Quantity", "Sales", "Discount", "Profit",    
    # Market Details
    "Market"
]
tempdf=tempdf[columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order.ID        50986 non-null  object 
 1   Order.Date      51085 non-null  object 
 2   weeknum         51290 non-null  int64  
 3   Year            51178 non-null  float64
 4   Ship.Date       50917 non-null  object 
 5   Order.Priority  51157 non-null  object 
 6   Ship.Mode       50822 non-null  object 
 7   Customer.ID     51064 non-null  object 
 8   Customer.Name   50864 non-null  object 
 9   Segment         50863 non-null  object 
 10  City            51044 non-null  object 
 11  State           50875 non-null  object 
 12  Country         50883 non-null  object 
 13  Region          50879 non-null  object 
 14  Product.ID      51000 non-null  object 
 15  Product.Name    51259 non-null  object 
 16  Category        51171 non-null  object 
 17  Sub.Category    51014 non-null 

Cleaning the dataset:

In [6]:
# Changing Datatype of Object to String for the concerning factors
for i in tempdf.columns:
    if(tempdf[i].dtypes=='object'):
        tempdf[i]=tempdf[i].astype("string")
tempdf['Order.Date']=pd.to_datetime(tempdf['Order.Date'])
tempdf['Ship.Date']=pd.to_datetime(tempdf['Ship.Date'])

#setting the dataframe index to show number of observation and sorting w.r.t. Order.Date
tempdf=tempdf.sort_values(by='Order.Date', ascending=False)
tempdf.reset_index()
tempdf.set_index(np.arange(0,len(tempdf)), inplace=True)

In [7]:
# Discarding observations where the shipping date is earlier than the order date
before_cleaing=len(tempdf)
tempdf = tempdf[tempdf['Ship.Date'] >= tempdf['Order.Date']]

# Checking for duplicate entries
print(f"Number of Duplicate rows: {df.duplicated().sum()}")
print(f"Number of Observations after removing duplicates and rows with wrong Shipping Date: {len(tempdf)}")
print(f"Data lost after the above step: {round((before_cleaing-len(tempdf))/before_cleaing*100, 2)}%")

Number of Duplicate rows: 0
Number of Observations after removing duplicates and rows with wrong Shipping Date: 50715
Data lost after the above step: 1.12%


Handling the missing values

In [8]:
#Market and Market2
M1=tempdf["Market"].dropna().unique().tolist()
M2=tempdf["Market2"].dropna().unique().tolist()
M2.append("North America")

for i in range(len(M1)): 
    tempdf.loc[tempdf["Market"].isna() & (tempdf["Market2"]==M2[i]), "Market"] = M1[i]
    tempdf.loc[tempdf["Market2"].isna() & (tempdf["Market"]==M1[i]), "Market2"] = M2[i]

In [ ]:
# Check for missing values in 'Customer.ID'
missing_id = tempdf[tempdf['Customer.ID'].isna()]

# Find unique customer information in the DataFrame
unique_id = tempdf[['Customer.ID', 'City', 'State', 'Country', 'Region', 'Customer.Name']].dropna()

# Create a mapping for missing 'Customer.Name'
mapping = unique_id.set_index(['Customer.Name', 'City', 'State', 'Country', 'Region'])['Customer.ID'].to_dict()

# Apply the mapping to fill missing values in 'Customer.Name'
tempdf['Customer.ID'] = tempdf.apply(
    lambda row: mapping.get((row['Customer.Name'], row['City'], row['State'], row['Country'], row['Region']), row['Customer.ID']),
    axis=1
)
print(tempdf['Customer.ID'].isna().sum())

In [ ]:
# Check for missing values in 'Customer.Name'
missing_names = tempdf[tempdf['Customer.Name'].isna()]

# Find unique customer information in the DataFrame
unique_customers = tempdf[['Customer.ID', 'City', 'State', 'Country', 'Region', 'Customer.Name']].dropna()

# Create a mapping for missing 'Customer.Name'
mapping = unique_customers.set_index(['Customer.ID', 'City', 'State', 'Country', 'Region'])['Customer.Name'].to_dict()

# Apply the mapping to fill missing values in 'Customer.Name'
tempdf['Customer.Name'] = tempdf.apply(
    lambda row: mapping.get((row['Customer.ID'], row['City'], row['State'], row['Country'], row['Region']), row['Customer.Name']),
    axis=1
)

# Check the result
print(tempdf['Customer.Name'].isna().sum())

In [9]:
#Location data
location=tempdf.loc[:,['City', 'State', 'Country', 'Region']].drop_duplicates()
loc_u=location.dropna()
tempdf.isna().sum()

Order.ID          301
Order.Date          0
weeknum             0
Year              110
Ship.Date           0
Order.Priority    133
Ship.Mode         463
Customer.ID       224
Customer.Name     423
Segment           424
City              244
State             412
Country           403
Region            410
Product.ID        285
Product.Name       29
Category          116
Sub.Category      275
Quantity          147
Sales             177
Discount          459
Profit            323
Market              0
Market2             0
dtype: int64

In [10]:
# Merge tempdf with location on 'City' and 'State'
merged_df_Country = tempdf.merge(loc_u, on=['City', 'State', 'Region'], how='left', suffixes=('', '_loc'))
merged_df_State = tempdf.merge(loc_u, on=['Country', 'City',  'Region'], how='left', suffixes=('', '_loc'))
#merged_df_City = tempdf.merge(loc_u, on=['Country', 'State', 'Region'], how='left', suffixes=('', '_loc'))
#merged_df_Region = tempdf.merge(loc_u, on=['Country', 'State', 'City'], how='left', suffixes=('', '_loc'))


# Fill missing 'Country' values in tempdf with the values from loc_u
tempdf['Country'] = tempdf['Country'].fillna(merged_df_Country['Country_loc'])
tempdf['State'] = tempdf['State'].fillna(merged_df_State['State_loc'])
#tempdf['City'] = tempdf['City'].fillna(merged_df_City['City_loc'])
#tempdf['Region'] = tempdf['Region'].fillna(merged_df_Region['Region_loc'])


# Check for missing values in 'Region'
missing_region = tempdf[tempdf['Region'].isna()]
unique_regions = tempdf[['Country', 'Region']].dropna()
mapping = unique_regions.set_index(['Country'])['Region'].to_dict()
tempdf['Region'] = tempdf.apply(
    lambda row: mapping.get((row['Country']), row['Region']),
    axis=1
)


print(tempdf.isna().sum(), len(tempdf))


Order.ID          301
Order.Date          0
weeknum             0
Year              110
Ship.Date           0
Order.Priority    133
Ship.Mode         463
Customer.ID       224
Customer.Name     423
Segment           424
City              244
State              14
Country            10
Region              9
Product.ID        285
Product.Name       29
Category          116
Sub.Category      275
Quantity          147
Sales             177
Discount          459
Profit            323
Market              0
Market2             0
dtype: int64 50715


In [11]:
#Order Date
#tempdf['Year'] = np.where(tempdf['Year'].isna(), pd.DatetimeIndex(tempdf['Order.Date']).year, tempdf['Year'])

In [12]:
before=len(tempdf)
Cleaned_df=tempdf.dropna(subset=['Sales','Profit', 'City'])
after=len(Cleaned_df)

print(f"Data lost after cleaning: {round((before-after)/before*100, 2)}%")
print(Cleaned_df.isna().sum(), len(Cleaned_df))


Data lost after cleaning: 1.46%
Order.ID          298
Order.Date          0
weeknum             0
Year                0
Ship.Date           0
Order.Priority    132
Ship.Mode         456
Customer.ID       220
Customer.Name     420
Segment           418
City                0
State              13
Country            10
Region              9
Product.ID        283
Product.Name       29
Category          114
Sub.Category      268
Quantity          145
Sales               0
Discount          451
Profit              0
Market              0
Market2             0
dtype: int64 49973


Saving the Cleaned DataFrame to a csv

In [ ]:
Cleaned_df['Segment']= Cleaned_df['Segment'].fillna(Cleaned_df['Segment'].mode()[0])
Cleaned_df['Ship.Mode']= Cleaned_df['Ship.Mode'].fillna(Cleaned_df['Ship.Mode'].mode()[0])
Cleaned_df['Order.Priority']= Cleaned_df['Order.Priority'].fillna(Cleaned_df['Order.Priority'].mode()[0])

<class 'pandas.core.frame.DataFrame'>
Index: 49973 entries, 0 to 51084
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order.ID        49675 non-null  string        
 1   Order.Date      49973 non-null  datetime64[ns]
 2   weeknum         49973 non-null  int64         
 3   Year            49973 non-null  float64       
 4   Ship.Date       49973 non-null  datetime64[ns]
 5   Order.Priority  49841 non-null  string        
 6   Ship.Mode       49517 non-null  string        
 7   Customer.ID     49753 non-null  string        
 8   Customer.Name   49553 non-null  string        
 9   Segment         49555 non-null  string        
 10  City            49973 non-null  string        
 11  State           49960 non-null  string        
 12  Country         49963 non-null  string        
 13  Region          49964 non-null  string        
 14  Product.ID      49690 non-null  string        
 15  Product